In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gamma, lognorm
import seaborn as sns
from datetime import date

In [ ]:
df = pd.read_table(
    "fits/rothman-sars_cov_2.tsv.gz", 
    converters={"date": date.fromisoformat},
    compression="gzip",
)

per_draw_df = pd.pivot_table(
    df,
    index='draws',
    values=['phi', 'ra_per_one_percent'],
)

## Inverse overdispersion, $\phi$

In [ ]:
# Prior
x = np.linspace(0, 10, 1000)
sns.lineplot(
    x = x,
    y = gamma.pdf(x, 2.0, scale=2.0),
    color = "black",
)

# Posterior
sns.histplot(
    data=per_draw_df,
    x="phi",
    stat="density",
    bins=40,
)

In [ ]:
# Prior
x = np.linspace(0.5e-7, 3.5e-7, 1000)
sns.lineplot(
    x = x,
    y = lognorm.pdf(x, 10, loc=-3),
    color = "black",
)

# Posterior
sns.histplot(
    data=per_draw_df,
    x="ra_per_one_percent",
    stat="density",
)

## Viral reads over time

In [ ]:
def posterior_grid(data, x, y, **kwargs):
    g = sns.relplot(
        data=data[
            (data["observation_type"] == "posterior") 
            & (data["draws"] < 9)
        ],
        x=x,
        y=y,
        col='draws',
        col_wrap=3,
        height=4,
        **kwargs,
    )
    g.set_titles("Posterior draw {col_name:1.0f}")

    ax = g.facet_axis(0, 0)
    for col in ax.collections:
        col.remove()
    sns.scatterplot(
        data=data[data["observation_type"] == "data"],
        x=x,
        y=y,
        ax=ax,
        legend=False,
        **kwargs,
    )
    ax.set_title("Observed", fontdict={"size": 10})
    return g

In [ ]:
posterior_grid(df, "date", "viral_reads", style="county", hue="plant")

## Prevalence vs time

In [ ]:
posterior_grid(df, "date", "prevalence_per100k", style="county", hue="plant")

In [ ]:
g = posterior_grid(
    df,
    "date",
    "prevalence_per100k",
    style="county",
    hue="plant",
)
g.set(yscale="log")

## Viral reads vs. prevalence

In [ ]:
posterior_grid(
    df,
    "prevalence_per100k",
    "viral_reads",
    style="county",
    hue="plant",
)